In [1]:
! pip install datasets transformers==4.26

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.36.0
    Uninstalling transformers-4.36.0:
      Successfully uninstalled transformers-4.36.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.14.3 requires transformers>=4.33.1, but you have transformers 4.26.0 which is incompatible.


In [2]:
# 설치된 Transformers lib의 버전이 최소 4.11.0 이상인지 확인합니다.
import transformers
print(transformers.__version__)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


4.26.0


In [3]:
# CUDA GPU가 사용 가능한지 확인합니다.
import torch
print(torch.cuda.is_available())
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1)

True


In [4]:
# Huggingface Dataset 레포지토리에서 GLUE-sst2 데이터를 로드합니다.
from datasets import load_dataset
train_dataset = load_dataset("glue", "sst2", split="train")
val_dataset = load_dataset("glue", "sst2", split="validation")

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


In [5]:
# train_dataset과 val_dataset의 크기를 확인합니다.
print(len(train_dataset))
print(len(val_dataset))

67349
872


In [6]:
# train_dataset과 val_dataset의 첫 번째 샘플을 출력합니다.
print(train_dataset[0])
print(val_dataset[0])

{'sentence': 'hide new secretions from the parental units ', 'label': 0, 'idx': 0}
{'sentence': "it 's a charming and often affecting journey . ", 'label': 1, 'idx': 0}


In [7]:
model_checkpoint = 'bert-base-uncased'

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint).to(device)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
def tokenize_function(examples):
    new_data = [s.replace('.','') for s in examples['sentence']]
    return tokenizer(new_data)

In [10]:
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True, num_proc=1, remove_columns=['sentence','idx','label'])
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True, num_proc=1, remove_columns=['sentence','idx','label'])
val_dataset_token_count_list = [len(t['input_ids']) for t in tokenized_val_dataset]
len(val_dataset_token_count_list)

  0%|          | 0/68 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

872

In [11]:
tokenized_train_dataset[0]

{'input_ids': [101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [12]:
tokenized_val_dataset[0]

{'input_ids': [101,
  2009,
  1005,
  1055,
  1037,
  11951,
  1998,
  2411,
  12473,
  4990,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
#block_size = tokenizer.model_max_length
block_size = 15

In [14]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples['input_ids'])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [15]:
lm_train_dataset = tokenized_train_dataset.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=1,
)

  0%|          | 0/68 [00:00<?, ?ba/s]

In [16]:
lm_val_dataset = tokenized_val_dataset.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=1,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [17]:
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-gluesst2",
    evaluation_strategy = "steps",
    learning_rate=1e-6,
    weight_decay=0.01,
    num_train_epochs=2,
    per_device_train_batch_size=8,
    report_to="none",
    push_to_hub=False,
    load_best_model_at_end = True
)

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_train_dataset,
    eval_dataset=lm_val_dataset,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

In [19]:
import os
os.environ["WANDB_WATCH"] = "false"

In [20]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 58606
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 7326
  Number of trainable parameters = 109514298
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
500,7.312900,5.534528
1000,4.912400,4.474309
1500,4.115500,3.717958
2000,3.460900,3.043533
2500,2.906500,2.443164
3000,2.432600,1.976239
3500,2.029800,1.634709
4000,1.749100,1.368657
4500,1.549100,1.159119
5000,1.376000,1.017090


***** Running Evaluation *****
  Num examples = 1394
  Batch size = 16
Saving model checkpoint to bert-base-uncased-finetuned-gluesst2/checkpoint-500
Configuration saved in bert-base-uncased-finetuned-gluesst2/checkpoint-500/config.json
Configuration saved in bert-base-uncased-finetuned-gluesst2/checkpoint-500/generation_config.json
Model weights saved in bert-base-uncased-finetuned-gluesst2/checkpoint-500/pytorch_model.bin
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 1394
  Batch size = 16
Saving model checkpoint to bert-base-uncased-finetuned-gluesst2/checkpoint-1000
Configuration saved in bert-base-uncased-finetuned-gluesst2/checkpoint-1000/config.json
Configuration saved in bert-base-uncased-finetuned-gluesst2

TrainOutput(global_step=7326, training_loss=2.5375777860824247, metrics={'train_runtime': 1402.8324, 'train_samples_per_second': 83.554, 'train_steps_per_second': 5.222, 'total_flos': 903830933142000.0, 'train_loss': 2.5375777860824247, 'epoch': 2.0})

In [21]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.10f}")

***** Running Evaluation *****
  Num examples = 1394
  Batch size = 16


Perplexity: 2.1628113266


In [22]:
predicted = []
from tqdm import tqdm

torch.cuda.empty_cache()
with torch.no_grad():
    dataset_count = len(tokenized_val_dataset)
    sentence_count = 0
    idx = 1
    predicted_cache = []
    for datum in tqdm(tokenized_val_dataset):
        input_vec = torch.Tensor(datum['input_ids']).to('cuda',dtype=torch.long).unsqueeze(0)
        output = model(input_ids=input_vec)
        logits = output.logits[0]
        len_vec = len(input_vec[0])
        for i in range(len_vec-1):
            l = torch.softmax(logits[i],dim=-1)[datum['input_ids'][i+1]].item()
            predicted_cache.append(l)
        if idx < dataset_count:
            predicted_cache.append(torch.softmax(logits[-1],dim=-1)[tokenized_val_dataset[idx]['input_ids'][0]].item())
            idx += 1
        predicted.append(sum(predicted_cache)/len(predicted_cache))

        # label.append(tokenizer.decode(datum['input_ids'],skip_special_tokens=True))
        # input_vec = torch.Tensor(datum['input_ids'][:-2]).to('cuda',dtype=torch.int32).unsqueeze(0)
        # generated = model.generate(input_vec,max_length=len(input_vec[0])+1,
        #                            no_repeat_ngram_size=2)
        # predicted_word.append(tokenizer.decode(generated[0],skip_special_tokens=True))

100%|██████████| 872/872 [00:13<00:00, 62.31it/s]


In [23]:
#export generated text to .csv file
import pandas

prw_df = pandas.DataFrame({"Expected":predicted})
prw_df.to_csv("./result2.csv",index_label='index')